In [75]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import os 

# Imports added from modifying the original code
import random
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, Column, join
from hetdex_tools.get_spec import get_spectra

Keras is a deep learning API written in Python, running on top of ther machine learning platform TensorFlow. Keras was developed with a focus on enabling fast experimentation.

So Keras is a high-level neural network library that runs on top of TensorFlow. Keras is more user friendly because it's built in Python.

In [2]:
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import History 

Comments and notes by me (Nick D) :) just to help me understand what's going on.

Callbacks: A callback is an object that can perform actions at various stages of training (e.g. at the start or end of an epoch, before or after a single batch, etc)

Callbacks can be used to:
   * Monitor metrics by writing TensorBoard logs
   * Periodically save model to disk drive
   * Do early stopping
   * Get internal states and stats during training
   * and more!

In [3]:
# This is creating a 'History' callback object. This object keeps track of the accuracy, loss and other training metrics
# for each epoch in the memory.
# Not 100% sure, but it seems like this is what allows the code to make plots later on.
history = History()

In [4]:
# Not entirely sure what the number means. But I'm assuming it's purely a naming convention and does not
# mean anything specific.
training_index = "31_cut"

### Opening HDR3 Catalog and converting it into a Df

In [5]:
# Opening HDR3 detections catalog and converting it into a pandas DF
# HDR3 is detections HETDEX found
# Using HDR3 NEP and not H20 NEP because we can make SN cuts!
HDR_source_cat = fits.open('/home/jovyan/Hobby-Eberly-Telesco/hdr3/catalogs/source_catalog_3.0.1.fits', memmap = True)
HDR3_data = HDR_source_cat[1].data
HDR3_DF = pd.DataFrame(HDR3_data, columns=HDR3_data.columns.names)

# Columns we will then take from the entire data set (it was huge so we needed to determine what we wanted to look at specifically).
# As the name suggests, these are the ones that are useful to us!
useful_hdr3_cols = ['source_id', 'detectid',  'selected_det', 'ra_mean', 'dec_mean', 'fwhm', 'shotid', 'field',  'ra', 'dec', 'wave', 'wave_err', 'flux', 'flux_err', 'sn', 'sn_err', 'chi2', 'chi2_err',
'linewidth', 'linewidth_err', 'plya_classification', 'z_hetdex', 'z_hetdex_conf', 'combined_plae']

# From the original DFs, taking the useful columns
reduced_hdr3_df = HDR3_DF.loc[:, useful_hdr3_cols]

### Filtering data

In [16]:
# Removing data from before 2017 because it isn't good (not useful to us)
# No need to do this for H20 NEP
removed_bad_shots_hdr3_df = reduced_hdr3_df[reduced_hdr3_df.shotid.values >= 20180000000]

# This will give high confidence detections. Something we would want to do also. What is sn threshold that Valentina's code is having trouble with.
# Reason why, we want high-confidence Lya. If we are very confident sn and another filter, then that's what we consider high-conf lya.
# Once noise and high-confidence sample. We can start exanping on valentina's code and do our own stuff
hdr3_signal_to_noise_interval = removed_bad_shots_hdr3_df[removed_bad_shots_hdr3_df['sn'] > 6]


# Now I just take the sources stricly in the NEP field
hdr3_nep = hdr3_signal_to_noise_interval[hdr3_signal_to_noise_interval['field'] == 'nep']

Making skycoord object for hdr3 ra and dec

In [17]:
hdr3_skycoords = SkyCoord(hdr3_nep['ra'] * u.deg, hdr3_nep['dec'] * u.deg)

# QUESTIONS:
1. Would I have to check for duplicates before extracting? []

2. Do I care about spec_err for now? For now, no. [x]

3. How big should my random sample be? []

In [76]:
test = np.zeros(0)

In [77]:
test

array([], dtype=float64)

In [109]:
random_nep_LS = []

#random_nep_spectra = np.zeros(0)

for i in range(3):
    #pick a random coordinate from the skycoord object
    random_coord = hdr3_skycoords[random.randint(0, hdr3_skycoords.size)]
    # extract spectra at this random coordinate and append to list, can't append to np array because of 
    # data type issues will convert to array after loop!
    extraction = get_spectra(random_coord)
    x = extraction['spec'].value
    print("extraction['spec'].value", extraction['spec'].value)
    print("extraction['spec'].value.shape", extraction['spec'].value.shape)
    print('x', x)
    print("x[0].shape", x[0].shape)
    #print("np.squeeze(x, axis = 0)", np.squeeze(x, axis = 0))
    print("x.shape", x.shape)
    print("np.squeeze(x, axis = 0)", np.squeeze(x, axis = 0))
    print("np.squeeze(x, axis = 0).shape", np.squeeze(x, axis = 0).shape)
    print()
    #random_nep_LS.append(extraction['spec'].value)
    random_nep_LS.append(np.squeeze(extraction['spec'].value, axis = 0))
    #random_nep_spectra = np.append(random_nep_spectra, np.array([extraction['spec'].value]))
    
random_nep_spectra = np.array(random_nep_LS)

[INFO - 2022-11-11 05:59:30,570] Finding shots of interest
[INFO - 2022-11-11 05:59:36,733] Number of shots of interest: 8
[INFO - 2022-11-11 05:59:36,735] Extracting 8 sources
[INFO - 2022-11-11 05:59:36,922] Working on shot: 20210510026
[INFO - 2022-11-11 05:59:37,006] Working on shot: 20210605015
[INFO - 2022-11-11 05:59:37,018] Working on shot: 20210608014
[INFO - 2022-11-11 05:59:37,079] Working on shot: 20210610021
[INFO - 2022-11-11 05:59:37,118] Working on shot: 20210611014
[INFO - 2022-11-11 05:59:37,177] Working on shot: 20210612015
[INFO - 2022-11-11 05:59:37,250] Working on shot: 20210616020
[INFO - 2022-11-11 05:59:37,299] Working on shot: 20210616021
[INFO - 2022-11-11 05:59:38,716] Extracting 1
[INFO - 2022-11-11 05:59:39,204] Extraction of sources completed in 0.04 minutes.
[INFO - 2022-11-11 05:59:39,333] Retrieved 1 spectra.
[INFO - 2022-11-11 05:59:39,424] Finding shots of interest


extraction['spec'].value [[ 9.00642944  9.01742971  9.02839659 ... 12.92817222 12.93024977
  12.93230585]]
extraction['spec'].value.shape (1, 1036)
x [[ 9.00642944  9.01742971  9.02839659 ... 12.92817222 12.93024977
  12.93230585]]
x[0].shape (1036,)
x.shape (1, 1036)
np.squeeze(x, axis = 0) [ 9.00642944  9.01742971  9.02839659 ... 12.92817222 12.93024977
 12.93230585]
np.squeeze(x, axis = 0).shape (1036,)



[INFO - 2022-11-11 05:59:45,511] Number of shots of interest: 9
[INFO - 2022-11-11 05:59:45,513] Extracting 9 sources
[INFO - 2022-11-11 05:59:45,674] Working on shot: 20180805010
[INFO - 2022-11-11 05:59:45,735] Working on shot: 20180806010
[INFO - 2022-11-11 05:59:45,792] Working on shot: 20180807008
[INFO - 2022-11-11 05:59:45,841] Working on shot: 20180831003
[INFO - 2022-11-11 05:59:45,902] Working on shot: 20180904009
[INFO - 2022-11-11 05:59:45,962] Working on shot: 20180910008
[INFO - 2022-11-11 05:59:46,022] Working on shot: 20200425026
[INFO - 2022-11-11 05:59:46,074] Working on shot: 20200502022
[INFO - 2022-11-11 05:59:46,131] Working on shot: 20200518022
[INFO - 2022-11-11 05:59:47,364] Extracting 1
[INFO - 2022-11-11 05:59:47,800] Extraction of sources completed in 0.04 minutes.
[INFO - 2022-11-11 05:59:47,924] Retrieved 1 spectra.
[INFO - 2022-11-11 05:59:47,991] Finding shots of interest


extraction['spec'].value [[13.62506742 13.61212344 13.59916619 ... 34.87492323 34.87145585
  34.86802263]]
extraction['spec'].value.shape (1, 1036)
x [[13.62506742 13.61212344 13.59916619 ... 34.87492323 34.87145585
  34.86802263]]
x[0].shape (1036,)
x.shape (1, 1036)
np.squeeze(x, axis = 0) [13.62506742 13.61212344 13.59916619 ... 34.87492323 34.87145585
 34.86802263]
np.squeeze(x, axis = 0).shape (1036,)



[INFO - 2022-11-11 05:59:53,953] Number of shots of interest: 4
[INFO - 2022-11-11 05:59:53,954] Extracting 4 sources
[INFO - 2022-11-11 05:59:54,118] Working on shot: 20210608012
[INFO - 2022-11-11 05:59:54,173] Working on shot: 20210609014
[INFO - 2022-11-11 05:59:54,231] Working on shot: 20210611012
[INFO - 2022-11-11 05:59:54,285] Working on shot: 20210612013
[INFO - 2022-11-11 05:59:55,341] Extracting 1
[INFO - 2022-11-11 05:59:55,721] Extraction of sources completed in 0.03 minutes.
[INFO - 2022-11-11 05:59:55,830] Retrieved 1 spectra.


extraction['spec'].value [[ 3.98050247  3.98241521  3.98431945 ... 14.13570146 14.13704473
  14.13837403]]
extraction['spec'].value.shape (1, 1036)
x [[ 3.98050247  3.98241521  3.98431945 ... 14.13570146 14.13704473
  14.13837403]]
x[0].shape (1036,)
x.shape (1, 1036)
np.squeeze(x, axis = 0) [ 3.98050247  3.98241521  3.98431945 ... 14.13570146 14.13704473
 14.13837403]
np.squeeze(x, axis = 0).shape (1036,)



In [116]:
random_nep_spectra.shape

(3, 1036)

In [106]:
x.size

1036

In [78]:
random_nep_spectra[0].shape

(1, 1036)

In [79]:
x = random_nep_spectra[0]
np.squeeze(x, axis = 0).shape

(1036,)

In [84]:
random_nep_spectra

array([array([[3.3869852 , 3.3821485 , 3.37731035, ..., 4.68159778, 4.6813685 ,
               4.68114122]])                                                   ,
       array([[3.11184118, 3.11161769, 3.11138568, ..., 4.64897883, 4.64893582,
               4.64889315]])                                                   ,
       array([[2.03761393, 2.0388118 , 2.04000585, ..., 7.82602145, 7.82574789,
               7.82547692],
              [1.22775192, 1.22895395, 1.23014933, ..., 5.89813746, 5.89783571,
               5.8975368 ]])                                                   ],
      dtype=object)

In [87]:
y = random_nep_spectra

In [93]:
y

array([array([[3.3869852 , 3.3821485 , 3.37731035, ..., 4.68159778, 4.6813685 ,
               4.68114122]])                                                   ,
       array([[3.11184118, 3.11161769, 3.11138568, ..., 4.64897883, 4.64893582,
               4.64889315]])                                                   ,
       array([[2.03761393, 2.0388118 , 2.04000585, ..., 7.82602145, 7.82574789,
               7.82547692],
              [1.22775192, 1.22895395, 1.23014933, ..., 5.89813746, 5.89783571,
               5.8975368 ]])                                                   ],
      dtype=object)

In [92]:
y.shape

(3,)

In [ ]:
np.squeeze(y, ax)

Current problem: working on getting the array to a normal shape!

FIGURED IT OUT! Just need to use np.squeeze.

In [ ]:
# Loading in spectra samples
star_spec = np.load("/home/jovyan/work/stampede2/Nick_Capstone_Project/Valentina_samples/star_spec_reduced.npy") 
agn_spec = np.load("/home/jovyan/work/stampede2/Nick_Capstone_Project/Valentina_samples/all_agn_no_duplicates.npy")
lowz_spec = np.load("/home/jovyan/work/stampede2/Nick_Capstone_Project/Valentina_samples/lowz_no_duplicates.npy")
gal_spec = np.load("/home/jovyan/work/stampede2/Nick_Capstone_Project/Valentina_samples/gal_sample.npy")

In [ ]:
agn_spec[0]

In [ ]:
type(star_spec)

In [ ]:
# This function checks for nan values.
# Interestingly enough, when you do nan != nan, you get True. Becuase NaN is unequal with anything.
# But with numbers, you'll get false.
def isNaN(num):
    return num != num

In [ ]:
# If there are any values in the imports that are 'nan' (Not a Number) we remove them.
# Go through each value in matrix and check if NaN
def remove_nan(file):
    for i in range(0,len(file)):
        for j in range(0,len(file[i])):
            if isNaN(file[i][j]):
                # if value is nan, make the value equal to 0.0001
                # I'm assuming this was to avoid errors
                file[i][j]=0.00001

remove_nan(star_spec)
remove_nan(agn_spec)
remove_nan(lowz_spec)
remove_nan(gal_spec)

In [ ]:
# random.seed(2) will make the random numbers predictable, so random numbers, but the same random numbers from the same set/seed
# so I'll get the same random numbers everytime, until I change the seed
np.random.seed(2)
# random.shuffle will shuffle the values in an array or list. Like shuffling a deck of cards
np.random.shuffle(star_spec)

In [ ]:
# concatenate just joins arrays
all_images = np.concatenate((star_spec[::90],agn_spec,lowz_spec,gal_spec))
# making a training set
train_images = np.concatenate((star_spec[::90][:433],agn_spec[:444],lowz_spec[:837],gal_spec[:254]))
#train_images = agn_spec[:433]
#np.random.shuffle(train_images)

In [ ]:
# not sure what these val_images are, it seems to be the opposite half of the train_images.
val_images = np.concatenate((star_spec[::90][433:],agn_spec[444:],lowz_spec[837:],gal_spec[254:]))
#val_images = agn_spec[433:]
#np.random.shuffle(val_images)

In [ ]:
# This function will normalize the data.
# The idea is, there is such a large range of sources so it may be simpler for the algorithm to recognize sources
# if they are between 0 and 1.
# Dividing by the largest number preserves the information but scales it down.
# If we don't do this the sources could go from a very large number to a small one, which makes it harder for the algorithm
def Norm(data):
    new_data = []
    for spec in data:
        #print('spec_before', spec)
        spec = spec/spec.max()
        #print('spec_after', spec)
        #print('spec.max', spec.max())
        new_data.append(spec)
    return np.array(new_data)

In [ ]:
# Normalizing both data sets
train_images = Norm(train_images)
val_images = Norm(val_images)

In [ ]:
# Making cuts of the image sets and then normalizing them.
train_cut = train_images[:,88:1002]
val_cut = val_images[:,88:1002]
train_cut = Norm(train_cut)
val_cut = Norm(val_cut)

In [ ]:
latent_dim = 30
act = 'sigmoid'

In [ ]:
class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      #layers.InputLayer(input_shape=(1036)),
      layers.Dense(436, activation=act,name="e1"),
      #layers.Dropout(0.3),
      #layers.Dense(136, activation=act, name ='e2'),
      #BatchNormalization(),
      layers.Dropout(0.3),
      layers.Dense(latent_dim, activation=None, name = "e3"),
    ])
    self.decoder = tf.keras.Sequential([
      #layers.InputLayer(input_shape=(latent_dim,)),
      #layers.Dense(136, activation=act,name = "d1"),
      layers.Dense(436, activation=act,name = "d2"),
      #BatchNormalization(),
      layers.Dense(914, activation=None,name = "d3")
    ])#914 for cut. 1036 for all

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [ ]:
autoencoder = Autoencoder(latent_dim)
#autoencoder.build(input_shape=np.shape(train_images))
epoch = 500

In [ ]:
adg = tf.keras.optimizers.Adagrad(
    learning_rate=0.01)

In [ ]:
adam = tf.keras.optimizers.Adam(
    learning_rate=0.0005)

In [ ]:
def loss1(y_true, y_pred):
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    d = tf.reduce_mean(tf.math.square(y_pred - y_true), axis=-1)
    #tf.print(d, [d], "Inside loss function")
    #tf.print(d.shape)
    return d

autoencoder.compile(optimizer=adam, loss=loss1)#es.MeanSquaredError())

try:
    os.mkdir(f"models/training{training_index}")
    checkpoint_path = f"models/training{training_index}/cp_{training_index}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
except:
    checkpoint_path = f"models/training{training_index}/cp_{training_index}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
#autoencoder.summary()
#autoencoder.fit(train_images, train_images,
 #               epochs=10,
  #              shuffle=True,
   #             validation_data=(val_images,val_images))
hist = autoencoder.fit(train_cut, train_cut,
                epochs=epoch,
                shuffle=True,
                validation_data=(val_cut,val_cut),
                callbacks=[cp_callback])


In [ ]:
plt.figure()
x = np.linspace(1,epoch+1,epoch)
plt.plot(x,hist.history["loss"],label="loss")
plt.plot(x,hist.history["val_loss"],label="val loss")
plt.legend()
plt.savefig(f'Graphs/loss_{training_index}.png')
print(np.shape(train_images))

encoded_imgs = autoencoder.encoder(train_cut).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()
np.save(f"encoded_{training_index}",encoded_imgs)
np.save(f"decoded_{training_index}",decoded_imgs)

encoded_val = autoencoder.encoder(val_cut).numpy()
np.save(f"encoded_val_{training_index}",encoded_val)

In [ ]:
np.load("encoded_31_cut.npy")

In [ ]:
plt.figure()
plt.plot(train_cut[4])
plt.savefig("OG.png")
plt.figure()
plt.plot(decoded_imgs[4])
plt.savefig("RECON.png")
print(hist.history["val_loss"][-1])